In [90]:
#*--coding utf-8 --*
%run -i modulo_busca_arquivos.py
from ipywidgets import widgets, Box, VBox, HBox, Button, Layout, Checkbox, Text, Label
from IPython.display import HTML, Javascript, display

css = """
<style> 
.container{
        width: 98%;
    }

.output{
    height:100%;
    }

.output_scroll{
    height: 100%;
    }

.widget-label{
        height: 100px;
        white-space: wrap;
        font-weight: bold;
        font-size: 16px;
        text-align: center;
        }

.space{
    text-align: left;
    font-size: 13px;
    }

.vazio{
    color: red;
    font-size: 14px;
    text-align: left;
    }

.blue{
    color: blue;
    font-size: 14px;
    }

.vertical-line{
    border-right: 1px groove black;
    }
    
.horizontal-line{
    border-bottom: 1px groove black;
    }
</style>"""

emExibicao = widgets.Label(value="Exibindo "+str(len(t))+" de "+str(len(retorno))+" arquivos", description="Quantidade de Arquivos")
dev = []
train = []
decay = []
lr = []
hs = []
uniques = list()
only = list()
filtro = dict()
cells_visable = True

def toggle_code_cells(btn):
    global cells_visable
    if cells_visable:
        display(Javascript("$('div.input').hide();"))
        btn.description = "Mostrar Celulas"
    else:
        display(Javascript("$('div.input').show();"))
        btn.description = "Esconder Celulas"
    cells_visable = not cells_visable
    
toggle_cell = widgets.Button(description="Esconder Celulas", button_style="danger", layout=Layout(align_self="center", display="flex-wrap"))
toggle_cell.on_click(toggle_code_cells)

def recolheMenu(n):
    if(left_box.layout.visibility == "hidden"):
        vBox.layout.width = "85%"
        left_box.layout.visibility = "visible"
        left_box.layout.width = "25%"
    else:
        left_box.layout.visibility = "hidden"
        left_box.layout.width = "0%"
        vBox.layout.width = "100%"
           
toggle_filter = widgets.Button(description="Esconder Filtros", button_style="info", layout=Layout(align_self="center", display="flex-wrap"))
toggle_filter.on_click(recolheMenu)

def aplicaFiltros(n):
    global filtro
    global emExibicao
    for l in lr:
        if l.value == True:
            if not 'lr' in filtro:
                filtro['lr'] = set()
        
            filtro['lr'].add(l.description)
    for d in dev:
        if d.value == True:
            if not 'dev' in filtro:
                filtro['dev'] = set()
        
            filtro['dev'].add(d.description)
    for k in train:
        if k.value == True:
            if not 'train' in filtro:
                filtro['train'] = set()
            
            filtro['train'].add(k.description)
    for h in hs:
        if h.value == True:
            if not 'hidden_size' in filtro:
                filtro['hidden_size'] = set()
                
            filtro['hidden_size'].add(h.description)
    for dc in decay:
        if dc.value == True:
            if not 'decay' in filtro:
                filtro['decay'] = set()
        
            filtro['decay'].add(dc.description)
    
    z = applyFilters(filtro, retorno)

    
    if len(filtro) != 0:
        emExibicao = widgets.Label(value="Exibindo "+str(len(filtro))+" de "+str(len(t))+" arquivos", description="Quantidade de Arquivos")
    
filter_btn = widgets.Button(description="Aplicar Filtros", button_style="primary", layout=Layout(align_self="flex-end", display = "flex-wrap"))
filter_btn.on_click(aplicaFiltros)

box_layout = Layout(width="100%", 
                    display="flex",
                    flex_direction="column",
                    border="2px groove grey",
                    align_items='stretch')

vBox_layout = Layout(width="85%",
                    align_content="flex-start",
                    display="flex")

left_layout = Layout(width="25%", height="95%", flex_flow="column", display="flex", align_items="flex-start", align_self="center")
right_layout = Layout(width="99%", flex_flow="row", justify_content="space-between", display="flex", align_items="flex-end", align_self="center")
main_layout = Layout(width="99%", height="100%")

filter_btn = widgets.Button(description="Aplicar Filtros", button_style="primary", layout=Layout(align_self="flex-end", display = "flex-wrap"))
filter_btn.on_click(aplicaFiltros)

def buildUniKey(key):
    for i in key:
        if cmp(i, 'lr') == 0:
            buildLr('u')
        elif cmp(i, 'train') == 0:
            buildTrain('u')
        elif cmp(i, 'hidden_size') == 0:
            buildHs('u')
        elif cmp(i, 'dev') == 0:
            buildDev('u')
        elif cmp(i, 'decay') == 0:
            buildDecay('u')

def buildMultKey(key):
    for i in key:
        if cmp(i, 'lr') == 0:
            buildLr('m')
        elif cmp(i, 'train') == 0:
            buildTrain('m')
        elif cmp(i, 'hidden_size') == 0:
            buildHs('m')
        elif cmp(i, 'dev') == 0:
            buildDev('m')
        elif cmp(i, 'decay') == 0:
            buildDecay('m')

def buildLr(tipo):
    global lr
    if cmp(tipo, 'u') == 0:
        lr = [widgets.Label(description=str(t['unique']['lr']), value="Learnig Rate: "+str(t['unique']['lr']),
                       width="95%",
                       style={'description_width': 'initial'}).add_class('space').add_class('blue')]
    elif cmp(tipo, 'm') == 0:
        lr = [Checkbox(description=str(l), 
                       width="100%",
                       style={'description_width': 'initial'}) for l in t['multiple']['lr']]

def buildTrain(tipo):
    global train
    if cmp(tipo, 'u') == 0:
        train = [widgets.Label(description=str(t['unique']['train']), value="Train: "+str(t['unique']['train']),
                          width="95%",
                          style={'description_width': 'initial'}).add_class('space').add_class('blue')]
        
    elif cmp(tipo, 'm') == 0:
        train = [Checkbox(description=r,
                          width="100%",
                          style={'description_width': 'initial'}) for r in t['multiple']['train']]
        
def buildHs(tipo):
    global hs
    if cmp(tipo, 'u') == 0:
        hs = [widgets.Label(description=str(t['unique']['hidden_size']), value="Hidden_Size: "+str(t['unique']['hidden_size']),
                       width="95%",
                       style={'description_width': 'initial'}).add_class('space').add_class('blue')]
        
    elif cmp(tipo, 'm') == 0:
        hs = [Checkbox(description=str(z),
                       width="100%",
                       style={'description_width': 'initial'}) for z in t['multiple']['hidden_size']]

def buildDev(tipo):
    global dev
    if cmp(tipo, 'u') == 0:
        dev = [widgets.Label(description=t['unique']['dev'], value="Dev: "+t['unique']['dev'], width="95%",
                        style={'description_width': 'initial'}).add_class('space').add_class('space').add_class('blue')]
        
    elif cmp(tipo, 'm') == 0:
        dev = [Checkbox(description=w,
                        width="100%",
                        style={'description_width': 'initial'}) for w in t['multiple']['dev']]
        
def buildDecay(tipo):
    global decay
    if cmp(tipo, 'u') == 0:
        decay = [widgets.Label(value="Decay: "+t['unique']['decay'], description=t['unique']['decay'], width="95%").add_class('space').add_class('blue')]
        
    elif cmp(tipo, 'm') == 0:
        decay = [Checkbox(description=w,
                        width="100%",
                        style={'description_width': 'initial'}) for w in t['multiple']['decay']]


uniKey = t['unique'].keys()
multKey = t['multiple'].keys()
if len(uniKey) > 0:
    buildUniKey(uniKey)
        
if len(multKey) > 0:
    buildMultKey(multKey)
        
def getUnique():
    global uniques, only
    if len(decay) == 1:
        uniques.extend(decay)
        only.append('decay')
    if len(lr) == 1:
        uniques.extend(lr)
        only.append('lr')
    if len(dev) == 1:
        uniques.extend(dev)
        only.append('dev')
    if len(train) == 1:
        uniques.extend(train)
        only.append('train')
    if len(hs) == 1:
        uniques.extend(hs)
        only.append('hidden_size')
    
    return uniques

## Criação dos Labels que serão exibidos para informar quais as propriedas que possuem filtros que podem ser aplicados. A verificação abaixo avalia se, não houver nenhuma 
## propriedade comum entre os logs, na caixa de propriedades únicas deverá ser exibida a mensagem Não Há, para indicar que todas as propriedades são múltiplas

unicos = getUnique()
if len(unicos) == 0:
    vazio = widgets.Label(value="Nao Ha Caracteristicas Unicas", description="Nao ha",  display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class('vazio')
    unicos.append(vazio)
    
labelLr = widgets.Label(value="Learning Rate", description="Learnig Rate", display="flex-shrink", layout=Layout(width="95%")).add_class("space")
labelTrain = widgets.Label(value="Train", description="Train", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space")
labelHs = widgets.Label(value="Hidden Size", description="Hidden Size", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space")
labelDev = widgets.Label(value="Dev", description="Dev", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space")
labelDecay = widgets.Label(value="Decay", description="Decay", display="flex-shrink", layout=Layout(width="95%", align_self="center")).add_class("space")    

## Criação das Caixas que irão conter os valores possíveis para a aplicação dos filtros. A box static contém somente os valores que são comuns entre todos os arquivos e,logo, 
## não podem ser modificados

static = HBox([i for i in unicos],layout=Layout(width="95%", 
                               flex_flow="column",
                               display="inline-flex",
                               align_self="center")).add_class('horizontal-line')

boxLr = HBox([i for i in lr], layout=Layout(width="100%",
                                            flex_flow="column",
                                            display="inline-flex"))    

boxTrain = HBox([i for i in train], layout=Layout(width="100%",
                                                flex_flow="column",
                                                display="flex-shrink"))

boxHs = HBox([i for i in hs], layout=Layout(width="100%",
                                            flex_flow="column",
                                            display="flex-shrink"))

boxDev = HBox([i for i in dev], layout=Layout(width="100%",
                                              flex_flow="column",
                                              display="flex-shrink")) 

boxDecay = HBox([i for i in decay], layout=Layout(width="100%",
                                              flex_flow="column",
                                              display="flex-shrink")) 


## Verificando se algum dos itens é comum a todos os logs. Caso seja, a box e o label não deverá ser exibido, pois não poderá ser filtrado e será apresentado na box static
if 'lr' in unicos:
    boxLr.layout.display = 'none'
    labelLr.layout.display = "none"
    
if 'train' in only:
    boxTrain.layout.display = 'none'
    labelTrain.layout.display = 'none'

if 'hidden_size' in only:
    boxHs.layout.display = 'none'
    labelHs.layout.display = 'none'
    
if 'dev' in only:
    boxDev.layout.display = 'none'
    labelDev.layout.display = 'none'
    
if 'decay' in only:
    boxDecay.layout.display = 'none'
    labelDecay.layout.display = 'none'


## left_box é a box onde os todos os filtros serão exibidos, e será posicionada ao lado esquerdo da tela de aplicação.

left_box = VBox([widgets.Label(value="Filtros", description="Filtros", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                 widgets.Label(value="Caracteristicas Unicas", description="Caracteristicas Unicas", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                 static,
                 widgets.Label(value="Caracteristicas Multiplas", description="Caracteristicas Multiplas", display="flex-shrink", layout=Layout(width="100%", align_self="center")),
                 labelLr,
                 boxLr,
                 labelTrain,
                 boxTrain,
                 labelHs,
                 boxHs,
                 labelDev,
                 boxDev,
                 labelDecay,
                 boxDecay,
                 filter_btn]).add_class('vertical-line')

left_box.layout = left_layout

right_box = VBox(children=[toggle_filter, emExibicao, toggle_cell]).add_class('horizontal-line')
right_box.layout=right_layout

mainBox = VBox(children=[])
mainBox.layout=main_layout

vBox = VBox(children=[right_box, mainBox])
vBox.layout = vBox_layout

box = Box(children=[left_box, vBox])
box.layout = box_layout


display(HTML(css))
box

Insira o nome da pasta que deseja abrir os logs: 
/home/junior/PycharmProjects/lia-pln-notebooks-master


Box(children=(VBox(children=(Label(value=u'Filtros', description=u'Filtros', layout=Layout(align_self=u'center', width=u'100%')), Label(value=u'Caracteristicas Unicas', description=u'Caracteristicas Unicas', layout=Layout(align_self=u'center', width=u'100%')), HBox(children=(Label(value=u'Decay: none', description=u'none', _dom_classes=('space', 'blue')),), layout=Layout(align_self=u'center', display=u'inline-flex', flex_flow=u'column', width=u'95%'), _dom_classes=('horizontal-line',)), Label(value=u'Caracteristicas Multiplas', description=u'Caracteristicas Multiplas', layout=Layout(align_self=u'center', width=u'100%')), Label(value=u'Learning Rate', description=u'Learnig Rate', layout=Layout(width=u'95%'), _dom_classes=('space',)), HBox(children=(Checkbox(value=False, description=u'0.005', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'0.01', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'0.015', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'0.02', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'0.05', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'0.1', style=DescriptionStyle(description_width=u'initial'))), layout=Layout(display=u'inline-flex', flex_flow=u'column', width=u'100%')), Label(value=u'Train', description=u'Train', layout=Layout(align_self=u'center', width=u'95%'), _dom_classes=('space',)), HBox(children=(Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetBahiaPartial100Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetBahiaPartial10Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetBahiaPartial50Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSantosPartial100Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSantosPartial10Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSantosPartial500Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSantosPartial50Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSaoPauloPartial1000Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSaoPauloPartial100Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSaoPauloPartial10Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSaoPauloPartial500Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoSetSaoPauloPartial50Over.txt', style=DescriptionStyle(description_width=u'initial')), Checkbox(value=False, description=u'/home/jonatas/Documentos/DataSetCreate/overSamples/especial/FifthtrainTwoS

0
1
2
3
4
5
{'dev': ['/home/junior/PycharmProjects/lia-pln-notebooks-master/shortdoc_class/distant_supervision/log/wnn-lr_0.01.log']}
